In [10]:
import pandas as pd
from assnat.clean import complete_preproc
from assnat.params import *
from pprint import pprint
import functools

import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, CamembertForMaskedLM, AutoTokenizer, AutoConfig
from sklearn.metrics import confusion_matrix, f1_score

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm.notebook import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
!pip install transformers plotly==5.8.0 pyyaml==5.4.1 datasets pytorch-lightning > /dev/null 2>&1

In [4]:
camembert = CamembertForMaskedLM.from_pretrained('camembert-base')

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
tokenizer = AutoTokenizer.from_pretrained('camembert-base')

In [6]:
tokenizer('My tokenizers and model must match')

{'input_ids': [5, 4646, 1200, 6840, 23038, 10, 1168, 16320, 19046, 1136, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
data1 = pd.read_csv('data/leg15.csv')
data2 = pd.read_csv('data/leg16.csv')
df = pd.concat([data1, data2], ignore_index=True, axis=0)
df = complete_preproc(df, drop_fam= ['Variable'], na_col = ['Texte','famille'], drop_names = ['Mme la présidente', 'M. le président'], min_words = 20, punct_opt= True)

Columns dropped
family dropped
NaN dropped
Names dropped
Short sentences removed
Preprocessing done!


In [11]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df.famille)
X_train,X_test,Y_train,Y_test = train_test_split(list(df['Texte']),y,test_size=0.2,random_state=28)


In [13]:
df['Texte'] = df['Texte'].values.astype('str')

In [14]:
train_input = tokenizer(X_train, tokenizer)
test_input = tokenizer(X_test, tokenizer)

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).